In [1]:
import json
import string as s
import pandas as pd
import os.path,inspect, re 
import random as r
from types import SimpleNamespace
import sys
import pathlib
import time
import os
import zipfile
import concurrent.futures

from pyspark.sql import types as T,functions as F
from pyspark.sql.window import Window
from datetime import datetime, timedelta 
import pyspark
from pyspark import SparkContext ,SparkConf, SparkSession, SparkContext
from pyspark.sql import SQLContext  

In [5]:
conf = ( SparkConf()
        .setMaster("local")
        .setAppName("MyApp")
        .set("spark.executor.memory","6g")
        .set("spark.sql.session.timeZone", "America/Sao_Paulo")
        .set('spark.sql.execution.arrow.pyspark.enabled' , False)
        .set('spark.sql.caseSensitive', True))
        
sc = SparkContext(conf=conf)

In [7]:

class DirSet:
    def __init__(self, directory=None):
        self.dir = directory
        
    def countDir(self):
        
        dcFiles = dbutils.fs.ls(self.dir)
        df = spark.createDataFrame(sc.parallelize(dcFiles))
        count = df.count()
        displayHTML("DIRECTORY SET AS: ...  %s" % self.dir)
        displayHTML("TOTAL OF FILES FOUND AFTER SCANNING : ... %s" % count)   

    def graphDir(self):
        
        dcFiles = dbutils.fs.ls(self.dir)
        df = spark.createDataFrame(sc.parallelize(dcFiles))
        df = df.sort(df.size)
        count = df.count()
        display(df)


    def descDir(self):
        #origin
        dcFiles = dbutils.fs.ls(self.dc)
        df = spark.createDataFrame(sc.parallelize(dcFiles))
        df = df.describe()
        display(df)

    def filesDir(self):
        #origin
        dcFiles = dbutils.fs.ls(self.dir)
        return dcFiles



1
